In [1]:
from torch_geometric.datasets import PCPNetDataset
from torch_geometric.transforms import ToSparseTensor, KNNGraph, Compose
from normal_diffusion.data.transforms import DistanceToEdgeWeight, KeepNormals

# Choose the root directory where you want to save the dataset
root = "../data/PCPNetDataset"

# Download and load the training dataset
dataset = PCPNetDataset(
    root=root,
    category="NoNoise",
    split="train",
    transform=Compose([KNNGraph(k=6), DistanceToEdgeWeight(), ToSparseTensor(), KeepNormals()]),
)

print(len(dataset))
first_collection = dataset[0]
print(first_collection.x.shape)
print(first_collection.adj_t)
print(first_collection)

8
torch.Size([100000, 3])
SparseTensor(row=tensor([    0,     0,     0,  ..., 99999, 99999, 99999]),
             col=tensor([52139, 56092, 58051,  ..., 51001, 82571, 99218]),
             val=tensor([0.1673, 0.1674, 0.1656,  ..., 0.1664, 0.1682, 0.1650]),
             size=(100000, 100000), nnz=600000, density=0.01%)
Data(x=[100000, 3], pos=[100000, 3], test_idx=[5000], adj_t=[100000, 100000, nnz=600000])


In [2]:
import torch
from normal_diffusion.models import GCNModel
model = GCNModel()
t = torch.ones(first_collection.x.shape[0])
predicted_normals = model(graph_data=first_collection, t=t)
print(predicted_normals)


tensor([[ 0.9543,  4.7256,  0.4208],
        [ 1.3620,  3.5609,  0.0548],
        [ 0.5082,  4.5813,  0.4453],
        ...,
        [ 1.3655,  4.0132, -0.3092],
        [ 1.3941,  4.7888, -0.8284],
        [ 1.1458,  4.3285,  0.1455]], grad_fn=<AddBackward0>)


/Users/nirg/courses/graphml/project/.venv/lib/python3.11/site-packages/torch_sparse/tensor.py:574: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:55.)
  return torch.sparse_csr_tensor(rowptr, col, value, self.sizes())


In [3]:
from diffusers import DDPMScheduler
from normal_diffusion.training.training import train_diffusion
scheduler = DDPMScheduler(num_train_timesteps=5, beta_schedule="squaredcos_cap_v2", clip_sample=False)
train_diffusion(model=model, dataloader=dataset[:1], scheduler=scheduler, n_epochs=100, lr=1e-3)

/Users/nirg/courses/graphml/project/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/nirg/courses/graphml/project/.venv/lib/python3.11/site-packages/diffusers/configuration_utils.py:140: FutureWarning: Accessing config attribute `num_train_timesteps` directly via 'DDPMScheduler' object attribute is deprecated. Please access 'num_train_timesteps' over 'DDPMScheduler's config object instead, e.g. 'scheduler.config.num_train_timesteps'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


Epoch 10/100, Loss: -0.0836
Epoch 20/100, Loss: -0.3197
Epoch 30/100, Loss: -0.4619
Epoch 40/100, Loss: -0.5497
Epoch 50/100, Loss: -0.6265
Epoch 60/100, Loss: -0.6949
Epoch 70/100, Loss: -0.7501
Epoch 80/100, Loss: -0.7824
Epoch 90/100, Loss: -0.8083
Epoch 100/100, Loss: -0.8301
